In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import utm
from pyproj import Proj

pd.options.display.max_columns = None

In [2]:
df = pd.read_excel("../Data/2021_Accidentalidad.xlsx")
df.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14.0,Sin asistencia sanitaria,444976.463,4470508.259,N,NaN
1,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,NaN


# Duplicados

Eliminamos duplicados

In [3]:
df.drop_duplicates(inplace = True)

# Tipado de columnas

In [4]:
df.dtypes

num_expediente                  object
fecha                   datetime64[ns]
hora                            object
localizacion                    object
numero                          object
cod_distrito                   float64
distrito                        object
tipo_accidente                  object
estado_meteorológico            object
tipo_vehiculo                   object
tipo_persona                    object
rango_edad                      object
sexo                            object
cod_lesividad                  float64
lesividad                       object
coordenada_x_utm               float64
coordenada_y_utm               float64
positiva_alcohol                object
positiva_droga                 float64
dtype: object

Cambiamos el tipado de columnas que queremos

In [5]:
df['cod_distrito'] = pd.to_numeric(df["cod_distrito"], errors='coerce').astype('Int64')
df['cod_lesividad'] = pd.to_numeric(df["cod_lesividad"], errors='coerce').astype('Int64')
df["positiva_droga"] = df["positiva_droga"].astype("category")

In [6]:
df.shape

(39966, 19)

# Añadimos columnas

Añadimos columna de interés como el día de la semana

In [7]:
df['dia_semana'] = df['fecha'].dt.day_name()

In [8]:
df.head(1)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,dia_semana
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,Sin asistencia sanitaria,444976.463,4470508.259,N,NaN,Friday


# Eliminamos columnas

Eliminamos la columna de "positiva_droga" por tener la mayoría de campos en nulo y no aportar gran valor

In [9]:
df_limpio = df.drop(["positiva_droga"], axis = 1)

In [10]:
df_limpio.head(1)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday


# Gestión de Nulos

Eliminamos o sustituimos Nulos

In [11]:
df_limpio.isnull().sum()

num_expediente              0
fecha                       0
hora                        0
localizacion                0
numero                      3
cod_distrito                3
distrito                    3
tipo_accidente              4
estado_meteorológico     4156
tipo_vehiculo              54
tipo_persona                3
rango_edad                  0
sexo                        0
cod_lesividad           17489
lesividad               17489
coordenada_x_utm           12
coordenada_y_utm           12
positiva_alcohol          166
dia_semana                  0
dtype: int64

1️⃣ Columna "positiva_alcohol"

In [12]:
df_limpio["positiva_alcohol"].unique()

array(['N', 'S', nan], dtype=object)

In [13]:
df_limpio["positiva_alcohol"].value_counts()

N    38523
S     1277
Name: positiva_alcohol, dtype: int64

Sustituimos los nulos de la columna "positiva_alcohol" por "Sin especificar"

In [14]:
df_limpio['positiva_alcohol'] = df['positiva_alcohol'].fillna("Sin especificar")

2️⃣ Columna "tipo_accidente"

In [15]:
df_limpio["tipo_accidente"].unique()

array(['Colisión fronto-lateral', 'Colisión frontal',
       'Atropello a persona', 'Otro', 'Choque contra obstáculo fijo',
       'Alcance', 'Colisión lateral', 'Caída', 'Colisión múltiple',
       'Solo salida de la vía', 'Atropello a animal', 'Vuelco',
       'Despeñamiento', nan], dtype=object)

Sustituimos los nulos de la columna "tipo_accidente" por "Otro"

In [16]:
df_limpio['tipo_accidente'] = df['tipo_accidente'].fillna("Otro")

3️⃣ Columna "Tipo_vehiculo"

In [17]:
df_limpio["tipo_vehiculo"].unique()

array(['Turismo', 'Autobús', 'Motocicleta hasta 125cc', 'Todo terreno',
       'Bicicleta', 'Motocicleta > 125cc', 'Furgoneta', 'Camión rígido',
       'Sin especificar', 'Ciclomotor', 'Maquinaria de obras',
       'Tractocamión', 'Otros vehículos con motor', 'VMU eléctrico',
       'Cuadriciclo no ligero', 'Vehículo articulado', 'Autocaravana',
       'Bicicleta EPAC (pedaleo asistido)', 'Autobús articulado',
       'Autobus EMT', 'Ambulancia SAMUR', 'Cuadriciclo ligero',
       'Patinete', 'Ciclomotor de tres ruedas', 'Semiremolque',
       'Moto de tres ruedas > 125cc', 'Ciclo', 'Maquinaria agrícola',
       'Remolque', 'Moto de tres ruedas hasta 125cc',
       'Ciclomotor de dos ruedas L1e-B', 'Tren/metro',
       'Otros vehículos sin motor', 'Camión de bomberos',
       'Ciclo de motor L1e-A', 'Tranvía', 'Autobús articulado EMT', nan],
      dtype=object)

In [18]:
df_limpio["tipo_vehiculo"].value_counts() / df_limpio.shape[0] * 100

Turismo                              67.872692
Motocicleta hasta 125cc               7.478857
Furgoneta                             6.768253
Motocicleta > 125cc                   4.176050
Camión rígido                         2.422059
Autobús                               2.354501
Bicicleta                             1.728970
Ciclomotor                            1.571336
Todo terreno                          1.546314
VMU eléctrico                         1.441225
Tractocamión                          0.452885
Bicicleta EPAC (pedaleo asistido)     0.385328
Otros vehículos con motor             0.275234
Maquinaria de obras                   0.260221
Vehículo articulado                   0.195166
Autobús articulado                    0.180153
Autobus EMT                           0.170145
Sin especificar                       0.137617
Cuadriciclo ligero                    0.090077
Patinete                              0.087574
Ciclo                                 0.070060
Autocaravana 

Todos los tipos de vehículo con una representación muy baja los agrupamos en "Otros"

In [19]:
df_limpio ['tipo_vehiculo'] = df_limpio ['tipo_vehiculo']. replace ([
       'Sin especificar', 'Maquinaria de obras',
       'Tractocamión', 'Otros vehículos con motor',
       'Cuadriciclo no ligero', 'Vehículo articulado', 'Autocaravana',
       'Bicicleta EPAC (pedaleo asistido)', 'Autobús articulado',
       'Autobus EMT', 'Ambulancia SAMUR', 'Cuadriciclo ligero',
       'Patinete', 'Ciclomotor de tres ruedas', 'Semiremolque',
       'Moto de tres ruedas > 125cc', 'Ciclo', 'Maquinaria agrícola',
       'Remolque', 'Moto de tres ruedas hasta 125cc',
       'Ciclomotor de dos ruedas L1e-B', 'Tren/metro',
       'Otros vehículos sin motor', 'Camión de bomberos',
       'Ciclo de motor L1e-A', 'Tranvía', 'Autobús articulado EMT', np.nan], ["Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros", "Otros"])

4️⃣ Distrito

In [20]:
df_limpio["distrito"].unique()

array(['PUENTE DE VALLECAS', 'USERA', 'MORATALAZ', 'SALAMANCA',
       'SAN BLAS-CANILLEJAS', 'TETUÁN', 'CARABANCHEL', 'LATINA',
       'BARAJAS', 'HORTALEZA', 'VILLAVERDE', 'CIUDAD LINEAL',
       'FUENCARRAL-EL PARDO', 'CHAMBERÍ', 'CENTRO', 'MONCLOA-ARAVACA',
       'CHAMARTÍN', 'VILLA DE VALLECAS', 'RETIRO', 'ARGANZUELA',
       'VICÁLVARO', nan], dtype=object)

In [21]:
df_limpio["distrito"].isnull().sum()

3

In [22]:
df_limpio[df_limpio["distrito"].isnull()]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
25744,2021S015933,2021-09-11,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,<NA>,NaN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,<NA>,NaN,437390.155,4476258.031,N,Saturday
25745,2021S015933,2021-09-11,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,<NA>,NaN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,<NA>,NaN,437390.155,4476258.031,N,Saturday
25746,2021S015933,2021-09-11,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,<NA>,NaN,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 55 a 59 años,Mujer,14,Sin asistencia sanitaria,437390.155,4476258.031,N,Saturday


Sustituimos los nulos de distrito

In [23]:
df_limpio["distrito"].replace(np.nan, "Arganzuela", inplace = True)

In [24]:
df_limpio["cod_distrito"].replace(np.nan, "2", inplace = True)

5️⃣ Columna "Tipo_persona"

In [25]:
df_limpio["tipo_persona"].unique()

array(['Conductor', 'Pasajero', 'Peatón', nan], dtype=object)

In [26]:
df_limpio[df_limpio["tipo_persona"].isnull()]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
37104,2021S022518,2021-11-27,22:25:00,"AVDA. HISPANIDAD, 34",34,21,BARAJAS,Alcance,Despejado,Turismo,NaN,Desconocido,Desconocido,<NA>,NaN,451305.806,4479449.657,N,Saturday
40495,2021S024425,2021-12-21,16:30:00,AVENIDA MANUEL FRAGA IRIBARNE NUMERO 2,35,21,BARAJAS,Alcance,Despejado,Turismo,NaN,Menor de 5 años,Hombre,<NA>,NaN,448419.744,4481821.540,N,Tuesday
41575,2021S024983,2021-12-30,17:50:00,CALL. VALDETORRES DE JARAMA / CALL. ANGEL LUIS...,2,16,HORTALEZA,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,NaN,Desconocido,Desconocido,<NA>,NaN,445289.712,4480250.615,N,Thursday


Manipulamos la columna "Tipo_persona" para que es los nulos diga "sin especificar"

In [27]:
df_limpio["tipo_persona"].replace(np.nan, "Sin especificar", inplace = True)

In [28]:
#para ver culquier nulo de cualquier columna
#nan_rows = df_limpio[df_limpio.isnull().any(1)]
#nan_rows.head(10)

6️⃣ Columna "lesividad"

In [29]:
df_limpio["lesividad"].unique()

array(['Sin asistencia sanitaria',
       'Asistencia sanitaria sólo en el lugar del accidente', nan,
       'Ingreso inferior o igual a 24 horas',
       'Ingreso superior a 24 horas',
       'Atención en urgencias sin posterior ingreso',
       'Asistencia sanitaria inmediata en centro de salud o mutua',
       'Asistencia sanitaria ambulatoria con posterioridad',
       'Fallecido 24 horas'], dtype=object)

In [30]:
df_limpio["lesividad"].value_counts()

Sin asistencia sanitaria                                     12380
Asistencia sanitaria sólo en el lugar del accidente           5193
Ingreso inferior o igual a 24 horas                           1552
Atención en urgencias sin posterior ingreso                   1322
Asistencia sanitaria inmediata en centro de salud o mutua      947
Ingreso superior a 24 horas                                    552
Asistencia sanitaria ambulatoria con posterioridad             503
Fallecido 24 horas                                              28
Name: lesividad, dtype: int64

In [31]:
df_limpio['lesividad'] = df['lesividad'].fillna("Sin especificar")

 7️⃣ Columna "Coordinadas"

In [32]:
df_limpio[df_limpio["coordenada_x_utm"].isnull()]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
30817,2021S018891,2021-10-17,12:20:00,RONDA. SUR / CALL. MARTOS,241,13,PUENTE DE VALLECAS,Alcance,Despejado,Furgoneta,Conductor,Desconocido,Desconocido,<NA>,Sin especificar,NaN,NaN,N,Sunday
30818,2021S018891,2021-10-17,12:20:00,RONDA. SUR / CALL. MARTOS,241,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,Conductor,De 18 a 20 años,Hombre,<NA>,Sin especificar,NaN,NaN,N,Sunday
37939,2021S022950,2021-12-03,06:30:00,"CALL. REY FRANCISCO, 28",28,9,MONCLOA-ARAVACA,Colisión múltiple,Despejado,Otros,Conductor,Desconocido,Desconocido,<NA>,Sin especificar,NaN,NaN,N,Friday
37940,2021S022950,2021-12-03,06:30:00,"CALL. REY FRANCISCO, 28",28,9,MONCLOA-ARAVACA,Colisión múltiple,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,<NA>,Sin especificar,NaN,NaN,N,Friday
39443,2021S023850,2021-12-14,09:10:00,"AUTOV. M-23, KM 0,700",0,14,MORATALAZ,Colisión múltiple,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,7,Asistencia sanitaria sólo en el lugar del acci...,NaN,NaN,N,Tuesday
39444,2021S023850,2021-12-14,09:10:00,"AUTOV. M-23, KM 0,700",0,14,MORATALAZ,Colisión múltiple,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,14,Sin asistencia sanitaria,NaN,NaN,N,Tuesday
39445,2021S023850,2021-12-14,09:10:00,"AUTOV. M-23, KM 0,700",0,14,MORATALAZ,Colisión múltiple,Despejado,Turismo,Conductor,De 60 a 64 años,Hombre,14,Sin asistencia sanitaria,NaN,NaN,N,Tuesday
41231,2021S024803,2021-12-27,02:10:00,PLAZA. AGUSTIN GONZALEZ / CALL. SANTA GENOVEVA,1,15,CIUDAD LINEAL,Otro,NaN,Turismo,Conductor,De 45 a 49 años,Hombre,<NA>,Sin especificar,NaN,NaN,S,Monday
41425,2021S024915,2021-12-29,00:00:00,URB. MANZANA DE AZCA / CALL. AGUSTIN DE BETANC...,0,6,TETUÁN,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,Desconocido,Desconocido,<NA>,Sin especificar,NaN,NaN,N,Wednesday
41524,2021S024959,2021-12-29,13:55:00,CALL. GENERAL RICARDOS / CALL. BATALLA DE TORR...,220,11,CARABANCHEL,Alcance,Despejado,Otros,Conductor,De 45 a 49 años,Hombre,14,Sin asistencia sanitaria,NaN,NaN,N,Wednesday


8️⃣ Columna "estado_meteorológico"

In [33]:
df_limpio["estado_meteorológico"].unique()

array(['Despejado', 'Lluvia débil', 'Nublado', nan, 'Se desconoce',
       'Nevando', 'LLuvia intensa', 'Granizando'], dtype=object)

In [34]:
df_limpio["estado_meteorológico"].value_counts()

Despejado         30570
Nublado            2039
Lluvia débil       1999
LLuvia intensa      604
Se desconoce        393
Nevando             198
Granizando            7
Name: estado_meteorológico, dtype: int64

In [35]:
df_limpio['estado_meteorológico'] = df['estado_meteorológico'].fillna("Se desconoce")

# Manipulación de columnas

1️⃣ Distrito --> Poner el string en minúsculas con la primera letra mayúscula

In [36]:
df_limpio["distrito"] = df_limpio["distrito"].str.capitalize()
df_limpio.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday
1,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,Friday


2️⃣ Localización --> Poner el string en minúsculas con la primera letra mayúscula

In [37]:
df_limpio["localizacion"] = df_limpio["localizacion"].str.capitalize()
df_limpio.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
0,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday
1,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,Friday


# Eliminación de columnas

In [38]:
eliminar = ["cod_lesividad"]

In [39]:
df_limpio = df_limpio.drop(eliminar, axis = 1)

# Groupby

Agrupación por número de expediente

In [40]:
df_pie = pd.DataFrame(df_limpio.groupby(['num_expediente', 'distrito'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())

df_pie.head(1)

/var/folders/hb/9zbycmb97bx0n1b4ggqr8s_c0000gn/T/ipykernel_58415/4186470162.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_pie = pd.DataFrame(df_limpio.groupby(['num_expediente', 'distrito'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())


,,tipo_vehiculo,tipo_accidente,tipo_persona
num_expediente,distrito,,,
2020S019534,Puente de vallecas,5,5,5


In [41]:
df_pie.reset_index(inplace = True)
df_pie.head(2)

,num_expediente,distrito,tipo_vehiculo,tipo_accidente,tipo_persona
0,2020S019534,Puente de vallecas,5,5,5
1,2021S000001,Usera,5,5,5


In [42]:
df_pie.shape

(17952, 5)

Agrupación por distrito

In [43]:
df_pie_1 = pd.DataFrame(df_pie.groupby(['distrito'])["num_expediente"].count())

df_pie_1.head(2)

,num_expediente
distrito,
Arganzuela,844
Barajas,315


Agrupación por dia de la semana

In [44]:
df_pie_3 = pd.DataFrame(df_limpio.groupby(['num_expediente', 'dia_semana'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())

df_pie_3.head(1)

/var/folders/hb/9zbycmb97bx0n1b4ggqr8s_c0000gn/T/ipykernel_58415/572054545.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_pie_3 = pd.DataFrame(df_limpio.groupby(['num_expediente', 'dia_semana'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())


,,tipo_vehiculo,tipo_accidente,tipo_persona
num_expediente,dia_semana,,,
2020S019534,Friday,5,5,5


In [45]:
df_pie_3.reset_index(inplace = True)
df_pie_3.head(2)

,num_expediente,dia_semana,tipo_vehiculo,tipo_accidente,tipo_persona
0,2020S019534,Friday,5,5,5
1,2021S000001,Friday,5,5,5


In [46]:
df_pie_3.shape

(17952, 5)

In [47]:
df_pie_4 = pd.DataFrame(df_pie_3.groupby(['dia_semana'])["num_expediente"].count())

df_pie_4.head(2)

,num_expediente
dia_semana,
Friday,3099
Monday,2469


# Coordenadas

In [48]:
test_coordenada = utm.to_latlon(437390.155, 4476258.031, 30, 'T')
test_coordenada

(40.43460948264402, -3.738180465794323)

In [49]:
latitud =test_coordenada[0]

In [50]:
longitud = test_coordenada[1]

In [51]:
def utm_lat(lat, lon):
    try:
        coordenada = utm.to_latlon(lat, lon, 30, 'T')
        return coordenada[0]
    except:
        return None

In [52]:
def utm_lon(lat, lon):
    try:
        coordenada = utm.to_latlon(lat, lon, 30, 'T')
        return coordenada[1]
    except:
        return None

In [53]:
df_limpio["latitud"] = df_limpio.apply(lambda datos: utm_lat
                                           (datos["coordenada_x_utm"],
                                            datos["coordenada_y_utm"]), axis =1)

In [54]:
df_limpio["longitud"] = df_limpio.apply(lambda datos: utm_lon
                                           (datos["coordenada_x_utm"],
                                            datos["coordenada_y_utm"]), axis =1)

In [55]:
df_limpio.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana,latitud,longitud
0,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday,40.383349,-3.648245
1,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,Friday,40.383349,-3.648245
2,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,Sin especificar,444976.463,4470508.259,N,Friday,40.383349,-3.648245
3,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 10 a 14 años,Hombre,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday,40.383349,-3.648245
4,2020S019534,2021-01-01,04:30:00,Avda. pablo neruda / call. leoneses,57,13,Puente de vallecas,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday,40.383349,-3.648245


In [56]:
df_limpio["fecha"].unique()

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-11T00:00:00.000000000',
       '2021-01-12T00:00:00.000000000', '2021-01-13T00:00:00.000000000',
       '2021-01-14T00:00:00.000000000', '2021-01-15T00:00:00.000000000',
       '2021-01-16T00:00:00.000000000', '2021-01-17T00:00:00.000000000',
       '2021-01-18T00:00:00.000000000', '2021-01-19T00:00:00.000000000',
       '2021-01-20T00:00:00.000000000', '2021-01-21T00:00:00.000000000',
       '2021-01-10T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000',
       '2021-01-25T00:00:00.000000000', '2021-01-26T00:00:00.000000000',
       '2021-01-27T00:00:00.000000000', '2021-01-28

# Guardamos CSV

In [57]:
df_limpio.to_excel("../Data/df_limpio_transformado.xlsx")